# Basic infinite strength analysis

We are performing a very basic strength analysis on a plate with a hole coming from FEM mesh.

This contains the following steps:

* calculate von Mises equivalent stress
* perform a meanstress transformation
* perform a strength analysis against infinite strength data from a material

The infinite strength data consists of the median of the infinite strength and the scattering. So for a given local stress amplitude we can calculate the local failure probability. Comparing this failure probability with an tolerable failure probability of 1 ppm we can calculate the security factors of each element.

In [ ]:
import numpy as np
import pandas as pd

import pylife.stress.equistress
import pylife.strength.meanstress
import pylife.strength.infinite
import pylife.materialdata.woehler

import pylife.mesh.meshplot

import matplotlib.pyplot as plt
%matplotlib


In [ ]:
filename = 'plate_with_hole.h5'

stress = pd.read_hdf(filename, 'node_data')
stress['S13'] = np.zeros_like(stress['S11'])
stress['S23'] = np.zeros_like(stress['S11'])

 

In [ ]:
goodman = {
    'M': 0.5,
    'M2': 0.5/3.
}

R_goal = -1.


In [ ]:
woehler_data = {
    'strength_inf': 450.,
    'strength_scatter': 1.3
}

In [ ]:
result = (stress.groupby('element_id').mean()
          .equistress.mises().rename(columns={'mises': 'sigma_a'})
          .cyclic_stress.constant_R(0.0)
          .meanstress_mesh.FKM_goodman(pd.Series(goodman), R_goal)
          .infinite_security.factors(pd.Series(woehler_data), allowed_failure_probability=1e-6))

print("Lowest security factor:", result.security_factor.min())

In [ ]:
fig, ax = plt.subplots()
result[result['security_factor'] > 4.0] = 4.0
stress.join(result).meshplot.plot(ax, 'security_factor', cmap='jet_r')
plt.show()